In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv

if not load_dotenv():
    print("Failed to load .env file")

In [2]:
# We can do the same thing with a SQLite cache
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".claude.cache.db"))

In [3]:
from langgraph.graph import END
from claude_stonks_agent.graph import create_graph, AgentOutcomeStep

graph = create_graph()

question = 'what was the price of amazon at the start of the year?'
request = {'input': question}

last_state = None

for s in graph.stream(input=request):
    print(s)
    last_state = s

last_step = last_state[END]['steps'][-1]

match last_step:
    case AgentOutcomeStep():
        print(last_step.output)
    case _:
        print(f"Unexpected outcome: {last_step}")

{'entry': {'steps': [HumanInputStep("what was the price of amazon at the start of the year?")]}}
{'main': {'steps': [ToolCallStep("search_for_symbol(term=amazon)")]}}
{'tools': {'steps': [ToolCallResultStep("search_for_symbol(term=amazon) -> <results>
<result>
<symbol>AMZN</symbol>
<name>Amazon.com Inc</name>
</result>
<result>
<symbol>AZFL</symbol>
<name>Amazonas Florestal Ltd</name>
</result>
</results>
")]}}
{'main': {'steps': [ToolCallStep("current_date()")]}}
{'tools': {'steps': [ToolCallResultStep("current_date() -> 2024-02-25")]}}
{'main': {'steps': [ToolCallStep("price_at_date(symbol=AMZN, date=2024-01-01)")]}}
{'tools': {'steps': [ToolCallResultStep("price_at_date(symbol=AMZN, date=2024-01-01) -> 151.94")]}}
{'main': {'steps': [AgentOutcomeStep(So the price of Amazon stock at the start of 2024 on January 1st was $151.94.)]}}
{'__end__': {'input': 'what was the price of amazon at the start of the year?', 'steps': [HumanInputStep("what was the price of amazon at the start of the